# Estadísticas y aprendizaje automático sobre crímenes en San Francisco.
---
## Introducción

- **Autor**       : Juan Antonio Velasco Gómez
- **Contacto**    : [@juanvelasc0](https://twitter.com/juanvelasc0)
- **Blog**        : [Follow the white rabbit](https://www.fwhibbit.es/)

---
Algunas de las estadísticas obtenidas sobre crímenes en San Francisco desde 2010 a 2016.

![TotalCrimes](img/sf_total_crimes.jpg)

El siguiente notebook refleja algunas de las pruebas realizadas durante el proceso de aprendizaje de los diferentes paquetes de Python para el aprendizaje automático. El dataset que se utilizará a lo largo de estas pruebas refleja los crímenes de los últimos años en la ciudad de San Francisco. Dicho dataset es público (al igual que el de otras ciudades y países que pueden ser descargados para su uso).

En estas pruebas se hará uso de **sklearn** como librería principal de aprendizaje automático y será apoyada por otras como pandas, numpy o jupyter.

A lo largo del notebook se irán describiendo los pasos que se realizan sobre este conjunto de datos.


## San Francisco Crime Dataset

Lo primero será importar los paquetes necesarios para el programa. 

![Ecosistema](img/ecosys.png)

En esta prueba de concepto necesitaremos:

- **Numpy**: Numpy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays. 
- **Pandas**: Pandas is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series.
- **sklearn**: Scikit-Learn is a free software machine learning library for the Python programming language. It features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN, and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy.

In [1]:
# Importamos los diferentes paquetes que utilizaremos
import numpy as np
import pandas as pd
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

Disponemos de dos archivos CSV con los datos de entrenamiento y de test.

Vamos a leer dichos archivos y a guardarlos en nuestra variable correspondiente.

In [2]:
df_train = pd.read_csv('input/train.csv', parse_dates=['Dates'])
df_test = pd.read_csv('input/test.csv', parse_dates=['Dates'])

Los datos de entrenamiento (igual que los de test) tienen el siguiente formato:

In [3]:
print(df_train)

                     Dates                Category  \
0      2015-05-13 23:53:00                WARRANTS   
1      2015-05-13 23:53:00          OTHER OFFENSES   
2      2015-05-13 23:33:00          OTHER OFFENSES   
3      2015-05-13 23:30:00           LARCENY/THEFT   
4      2015-05-13 23:30:00           LARCENY/THEFT   
5      2015-05-13 23:30:00           LARCENY/THEFT   
6      2015-05-13 23:30:00           VEHICLE THEFT   
7      2015-05-13 23:30:00           VEHICLE THEFT   
8      2015-05-13 23:00:00           LARCENY/THEFT   
9      2015-05-13 23:00:00           LARCENY/THEFT   
10     2015-05-13 22:58:00           LARCENY/THEFT   
11     2015-05-13 22:30:00          OTHER OFFENSES   
12     2015-05-13 22:30:00               VANDALISM   
13     2015-05-13 22:06:00           LARCENY/THEFT   
14     2015-05-13 22:00:00            NON-CRIMINAL   
15     2015-05-13 22:00:00            NON-CRIMINAL   
16     2015-05-13 22:00:00                 ROBBERY   
17     2015-05-13 21:55:00  

![DataDescription](img/data_description.png)

Limpiamos dichos datos eliminando las variables que consideramos no son importantes para el entrenamiento

**NOTA: Estamos eliminando la variable "Resolution" puesto que se trata de un problema de regresión, si queremos que sea de clasificación podríamos dejarlo para multi-classification.**

In [4]:
df_train.drop(['Descript', 'Dates', 'Resolution', 'Address'], axis=1, inplace=True)
df_test.drop(['Dates', 'Address'], axis=1, inplace=True)

In [5]:
print("Datos de entrenamiento. \nTotal de elementos: " + str(len(df_train)) + "\n")
print(df_train)

Datos de entrenamiento. 
Total de elementos: 878049

                      Category  DayOfWeek  PdDistrict           X          Y
0                     WARRANTS  Wednesday    NORTHERN -122.425892  37.774599
1               OTHER OFFENSES  Wednesday    NORTHERN -122.425892  37.774599
2               OTHER OFFENSES  Wednesday    NORTHERN -122.424363  37.800414
3                LARCENY/THEFT  Wednesday    NORTHERN -122.426995  37.800873
4                LARCENY/THEFT  Wednesday        PARK -122.438738  37.771541
5                LARCENY/THEFT  Wednesday   INGLESIDE -122.403252  37.713431
6                VEHICLE THEFT  Wednesday   INGLESIDE -122.423327  37.725138
7                VEHICLE THEFT  Wednesday     BAYVIEW -122.371274  37.727564
8                LARCENY/THEFT  Wednesday    RICHMOND -122.508194  37.776601
9                LARCENY/THEFT  Wednesday     CENTRAL -122.419088  37.807802
10               LARCENY/THEFT  Wednesday     CENTRAL -122.419088  37.807802
11              OTHER O

In [6]:
print("Datos de test. \nTotal de elementos: " + str(len(df_test)) + "\n")
print(df_test)

Datos de test. 
Total de elementos: 884262

            Id  DayOfWeek  PdDistrict           X          Y
0            0     Sunday     BAYVIEW -122.399588  37.735051
1            1     Sunday     BAYVIEW -122.391523  37.732432
2            2     Sunday    NORTHERN -122.426002  37.792212
3            3     Sunday   INGLESIDE -122.437394  37.721412
4            4     Sunday   INGLESIDE -122.437394  37.721412
5            5     Sunday     TARAVAL -122.459024  37.713172
6            6     Sunday   INGLESIDE -122.425616  37.739351
7            7     Sunday   INGLESIDE -122.412652  37.739750
8            8     Sunday     MISSION -122.418700  37.765165
9            9     Sunday     CENTRAL -122.413935  37.798886
10          10     Sunday   INGLESIDE -122.408608  37.746787
11          11     Sunday     MISSION -122.411094  37.761048
12          12     Sunday     MISSION -122.411094  37.761048
13          13     Sunday     CENTRAL -122.402131  37.799364
14          14     Sunday     CENTRAL -12

Ahora dividiremos el conjunto de datos de entrenamiento en dos grupos

- Conjunto de datos de entrenamiento.
- Conjunto de datos de validación.

Para ello, dividiremos los índices en dos grandes grupos.

In [7]:
# Ahora dividiremos el conjunto de datos de entrenamiento en dos grupos
# De entrenamiento y de validacion
indices = np.arange(df_train.shape[0])
np.random.shuffle(indices)
train_indices = indices[:int(0.2 * df_train.shape[0])]
val_indices = indices[int(0.2 * df_train.shape[0]):]

Vamos a almacenar también los nombres de los posibles valores para las categorías de crímenes (para el etiquetado posterior)

In [8]:
col_names = np.sort(df_train['Category'].unique())
print(col_names)

['ARSON' 'ASSAULT' 'BAD CHECKS' 'BRIBERY' 'BURGLARY' 'DISORDERLY CONDUCT'
 'DRIVING UNDER THE INFLUENCE' 'DRUG/NARCOTIC' 'DRUNKENNESS'
 'EMBEZZLEMENT' 'EXTORTION' 'FAMILY OFFENSES' 'FORGERY/COUNTERFEITING'
 'FRAUD' 'GAMBLING' 'KIDNAPPING' 'LARCENY/THEFT' 'LIQUOR LAWS' 'LOITERING'
 'MISSING PERSON' 'NON-CRIMINAL' 'OTHER OFFENSES'
 'PORNOGRAPHY/OBSCENE MAT' 'PROSTITUTION' 'RECOVERED VEHICLE' 'ROBBERY'
 'RUNAWAY' 'SECONDARY CODES' 'SEX OFFENSES FORCIBLE'
 'SEX OFFENSES NON FORCIBLE' 'STOLEN PROPERTY' 'SUICIDE' 'SUSPICIOUS OCC'
 'TREA' 'TRESPASS' 'VANDALISM' 'VEHICLE THEFT' 'WARRANTS' 'WEAPON LAWS']


Y convertiremos las siguientes columnas en un formato numérico

In [9]:
df_train['Category'] = pd.Categorical(df_train['Category']).codes
df_train['DayOfWeek'] = pd.Categorical(df_train['DayOfWeek']).codes
df_train['PdDistrict'] = pd.Categorical(df_train['PdDistrict']).codes
df_test['DayOfWeek'] = pd.Categorical(df_test['DayOfWeek']).codes
df_test['PdDistrict'] = pd.Categorical(df_test['PdDistrict']).codes

In [10]:
print(df_train)

        Category  DayOfWeek  PdDistrict           X          Y
0             37          6           4 -122.425892  37.774599
1             21          6           4 -122.425892  37.774599
2             21          6           4 -122.424363  37.800414
3             16          6           4 -122.426995  37.800873
4             16          6           5 -122.438738  37.771541
5             16          6           2 -122.403252  37.713431
6             36          6           2 -122.423327  37.725138
7             36          6           0 -122.371274  37.727564
8             16          6           6 -122.508194  37.776601
9             16          6           1 -122.419088  37.807802
10            16          6           1 -122.419088  37.807802
11            21          6           8 -122.487983  37.737667
12            35          6           9 -122.412414  37.783004
13            16          6           4 -122.432915  37.784353
14            20          6           0 -122.397744  37

Guardamos en:
    
- **df_val**   : Elementos del grupo de entrenamiento (df_train) elegidos para validacion.
- **df_train** : Elementos del grupo de entrenamiento (df_train) elegidos para entrenamiento.   

In [11]:
df_val = df_train.ix[val_indices]
df_train = df_train.ix[train_indices]

/home/holdenv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


Ya disponemos de los datos en un formato correcto y listos para comenzar el entrenamiento. Para ello, comenzaremos creando un clasificador logístico y lo entrenaremos con dichos datos:

- **X_train** : Almacenará los valores de geolocalización (X,Y), el día de la semana y el distrito de policía de df_train para entrenamiento.
- **y_train** : Almacenará el valor de la categoría de cada uno de los crímenes de df_train para validación.
- **X_val**   : Almacenará los valores de geolocalización (X,Y), el día de la semana y el distrito de policía de df_train para validación.
- **y_val**   : Almacenará el valor de la categoría de cada uno de los crímenes de df_train para validación.


Vamos a dividir, por tanto, nuestras matrices de datos (df_train) y de validación (df_val) en dos partes:

In [12]:
# Construct the design matrix and response vector for the
# training data and the design matrix for the test data
from patsy import dmatrices, dmatrix
y_train, X_train = dmatrices('Category ~ X + Y + DayOfWeek + PdDistrict', df_train)
y_val, X_val = dmatrices('Category ~ X + Y + DayOfWeek + PdDistrict', df_val)

Y también vamos a guardar en X_test el correspondiente valor para df_test:

- **X_test** : Almacenará los valores de geolocalización (X,Y), el día de la semana y el distrito de policía de df_test.

In [13]:
X_test = dmatrix('X + Y + DayOfWeek + PdDistrict', df_test)

![PoliceLine](img/police_line.png)

In [14]:
# Crea el clasificador logico de regresion 
# y lo entrena usando los elementos X_train e y_train
logistic = LogisticRegression().fit(X_train, y_train.ravel())

In [15]:
print('Mean accuracy (Logistic): {:.4f}'.format(logistic.score(X_val, y_val.ravel())))

Mean accuracy (Logistic): 0.1989


In [16]:
# Realizamos las predicciones sobre el modelo entrenado
predict_probs = logistic.predict_proba(X_test)
print(predict_probs)

[[0.00270299 0.09722753 0.00063743 ... 0.08499983 0.03762308 0.01421035]
 [0.00270369 0.09721301 0.00063756 ... 0.08498773 0.03762042 0.01421994]
 [0.00189157 0.09149369 0.00052102 ... 0.06424135 0.04820271 0.00897871]
 ...
 [0.00233747 0.08668598 0.00059013 ... 0.06648313 0.04533993 0.01445907]
 [0.00272986 0.09433567 0.00068991 ... 0.08081158 0.04240819 0.01344561]
 [0.00134923 0.07234546 0.00039974 ... 0.03954361 0.05989965 0.00929881]]


In [17]:
# Mostramos los resultados de la prediccion sobre X_test
df_pred = pd.DataFrame(data=predict_probs, columns=col_names)
df_pred['Id'] = df_test['Id'].astype(int)
#df_pred.to_csv('output.csv', index=False)
print(df_pred)

           ARSON   ASSAULT  BAD CHECKS   BRIBERY  BURGLARY  \
0       0.002703  0.097228    0.000637  0.000389  0.047527   
1       0.002704  0.097213    0.000638  0.000389  0.047523   
2       0.001892  0.091494    0.000521  0.000270  0.043653   
3       0.002308  0.091079    0.000556  0.000323  0.043975   
4       0.002308  0.091079    0.000556  0.000323  0.043975   
5       0.001353  0.075237    0.000373  0.000180  0.035052   
6       0.002299  0.092919    0.000567  0.000325  0.044867   
7       0.002297  0.093264    0.000570  0.000325  0.045038   
8       0.002092  0.092632    0.000546  0.000298  0.044463   
9       0.002435  0.099516    0.000629  0.000357  0.048346   
10      0.002291  0.093897    0.000574  0.000326  0.045345   
11      0.002094  0.092515    0.000546  0.000298  0.044409   
12      0.002094  0.092515    0.000546  0.000298  0.044409   
13      0.002431  0.099754    0.000631  0.000357  0.048465   
14      0.002426  0.099903    0.000631  0.000357  0.048531   
15      

In [18]:
# Crea el clasificador logistico AdaBoost y lo entrena
# utilizando un árbol de decisión de maximo de profundidad (2)
# usando los datos de X_train e y_train
adaboosttree = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2))
adaboosttree.fit(X_train, y_train.ravel())
print('Mean accuracy (AdaBoost Logistic): {:.4f}'.format(adaboosttree.score(X_val, y_val.ravel())))

Mean accuracy (AdaBoost Logistic): 0.2017


In [19]:
# Realizamos las diferentes prediciones utilizando dicho clasificador AdaBoost entrenado
predict_probs_adaboosttree = adaboosttree.predict_proba(X_test)

In [20]:
# Mostramos los resultados de la prediccion sobre X_test
df_pred = pd.DataFrame(data=predict_probs_adaboosttree, columns=col_names)
df_pred['Id'] = df_test['Id'].astype(int)
print(df_pred)

           ARSON   ASSAULT  BAD CHECKS   BRIBERY  BURGLARY  \
0       0.027386  0.029298    0.026174  0.025410  0.028791   
1       0.028948  0.030396    0.027130  0.000821  0.029742   
2       0.026800  0.028408    0.027199  0.000141  0.028930   
3       0.026583  0.028719    0.025793  0.026583  0.028370   
4       0.026583  0.028719    0.025793  0.026583  0.028370   
5       0.027842  0.029959    0.026799  0.015600  0.029368   
6       0.026764  0.028942    0.025922  0.026298  0.028484   
7       0.026987  0.029057    0.024776  0.027416  0.028301   
8       0.026709  0.029045    0.025785  0.026729  0.028289   
9       0.027345  0.030223    0.028224  0.000157  0.029781   
10      0.027042  0.029233    0.025110  0.027454  0.028495   
11      0.026645  0.028863    0.024498  0.026540  0.028183   
12      0.026645  0.028863    0.024498  0.026540  0.028183   
13      0.027736  0.030373    0.028947  0.000145  0.030200   
14      0.029234  0.030113    0.029991  0.000128  0.030104   
15      

In [21]:
# Crea el clasificador RandomForest que utilizara un numero n
# de estimadores (árboles de decisión) y lo entrenamos de nuevo
# usando los datos de X_train e y_train
randforest = RandomForestClassifier(n_estimators=11)
randforest.fit(X_train, y_train.ravel())
print('Mean accuracy (Random Forest): {:.4f}'.format(randforest.score(X_val, y_val.ravel())))

Mean accuracy (Random Forest): 0.2213


In [22]:
# Realizamos las diferentes prediciones utilizando dicho clasificador RandomForest entrenado
predict_probs_randforest = randforest.predict_proba(X_test)

In [23]:
# Mostramos los resultados de la prediccion sobre X_test
df_pred = pd.DataFrame(data=predict_probs, columns=col_names)
df_pred['Id'] = df_test['Id'].astype(int)
#df_pred.to_csv('output.csv', index=False)
print(df_pred)

           ARSON   ASSAULT  BAD CHECKS   BRIBERY  BURGLARY  \
0       0.002703  0.097228    0.000637  0.000389  0.047527   
1       0.002704  0.097213    0.000638  0.000389  0.047523   
2       0.001892  0.091494    0.000521  0.000270  0.043653   
3       0.002308  0.091079    0.000556  0.000323  0.043975   
4       0.002308  0.091079    0.000556  0.000323  0.043975   
5       0.001353  0.075237    0.000373  0.000180  0.035052   
6       0.002299  0.092919    0.000567  0.000325  0.044867   
7       0.002297  0.093264    0.000570  0.000325  0.045038   
8       0.002092  0.092632    0.000546  0.000298  0.044463   
9       0.002435  0.099516    0.000629  0.000357  0.048346   
10      0.002291  0.093897    0.000574  0.000326  0.045345   
11      0.002094  0.092515    0.000546  0.000298  0.044409   
12      0.002094  0.092515    0.000546  0.000298  0.044409   
13      0.002431  0.099754    0.000631  0.000357  0.048465   
14      0.002426  0.099903    0.000631  0.000357  0.048531   
15      

![TOPCrimes](img/sf_top_crimes_map.png)

![Dashboard](img/sf_crime_dashboard.png)